In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import shutil

In [2]:
zerodhadirectory = r'C:\Projects\Finances\Zerodha TradeBook'

In [3]:

os.listdir(zerodhadirectory)

['dividends-XCP798-2022_2023.csv',
 'ledger-2022-23.csv',
 'tradebook-2021-22.csv',
 'tradebook-2022-23.csv',
 'tradebook-2023-24.csv',
 'zerodha_trades_2024-05-20.parquet']

In [5]:
trade_files = [
            file for file in os.listdir(zerodhadirectory)
            if file.endswith(".csv") and file.startswith("trade") 
        ]


In [6]:
trade_files

['tradebook-2021-22.csv', 'tradebook-2022-23.csv', 'tradebook-2023-24.csv']

In [16]:
trade_dfs = []
for file in trade_files:
    file_path = os.path.join(zerodhadirectory , file )
    df = pd.read_csv( file_path , header = 0)
    df['trade_value'] = df['quantity'] * df['price']
    # Select the desired columns
    desired_columns = ['symbol', 'isin', 'trade_date', 'exchange', 'segment', 'trade_type', 'quantity', 'price', 'trade_value']
    df = df[desired_columns]
    trade_dfs.append(df)

# Concatenate DataFrames and store the result back in the dictionary
try :
    combined_trade_df = pd.concat(trade_dfs, ignore_index=True)
except Exception as e:
    print(f'Failed to create to Zerodha trade df ::: \n {str(e)}')



In [24]:
# Get today's date
today_date = datetime.today()

# Convert the date to a string in the format "YYYY-MM-DD"
today_date_str = today_date.strftime("%Y-%m-%d")

zerodha_file_path = f"{zerodhadirectory}/zerodha_trades_{today_date_str}.parquet"

# Write the DataFrame to a Parquet file with the specified file name
try: 
    combined_trade_df.to_parquet(zerodha_file_path)
    print(f'\n Zerodha trade Parquet written !!')
except Exception as e:
    print(f'Failed to write Zerodha parquet ::: {zerodha_file_path} \n {str(e)}')


 Zerodha trade Parquet written !!


In [25]:
def file_backup(banksdirectory):
    # backup dir and create directory if not exists
    backup_dir = os.path.join (banksdirectory,'backup_parquets')
    os.makedirs(backup_dir , exist_ok= True)

    # Find parquet files
    files = [file for file in os.listdir(banksdirectory) if file.endswith(".parquet")]

    # Move files
    for file in files:
        source = os.path.join(banksdirectory, file)
        destination = os.path.join(backup_dir, file)
        shutil.move(source, destination)
        print(f"Moved {file} to backup directory")

'C:\\Projects\\Finances\\Zerodha TradeBook/zerodha_trades_2024-05-20.parquet'

### Parquet

In [8]:
zerodhadirectory = r'C:\\Projects\\Finances\\Zerodha TradeBook'
files = [file for file in os.listdir(zerodhadirectory) if file.endswith('.parquet')]
zerodha_file_path = os.path.join(zerodhadirectory, files[0])
zerodha_df = pd.read_parquet(zerodha_file_path)

In [9]:
zerodha_df.dtypes

symbol          object
isin            object
trade_date      object
exchange        object
segment         object
trade_type      object
quantity       float64
price          float64
trade_value    float64
dtype: object

In [28]:
columns = [ 'symbol' , 'trade_date', 'trade_type', 'trade_value']


In [50]:
zerodha_df.groupby(['symbol','trade_type'])[[ 'trade_value']].agg("sum")

trade_value
symbol     trade_type             
3MINDIA    buy            33149.95
           sell           36631.30
AARTIDRUGS buy              583.30
AARTIIND   buy            13336.55
           sell            9640.80
...                            ...
WHIRLPOOL  buy            11838.10
           sell            8676.85
WIPRO      buy             3832.70
           sell            2535.60
YATHARTH   sell           15831.55

[184 rows x 1 columns]

trade_value                                                   \
                 count          mean          std       min         25%   
symbol                                                                    
3MINDIA            2.0  34890.625000  2461.686193  33149.95  34020.2875   
AARTIDRUGS         1.0    583.300000          NaN    583.30    583.3000   
AARTIIND           8.0   2872.168750  2124.395786    891.60    965.6250   
AARTIPHARM         1.0    354.850000          NaN    354.85    354.8500   
ALKYLAMINE         4.0   3493.075000   425.491530   3195.00   3291.0750   
...                ...           ...          ...       ...         ...   
VGUARD            12.0   2220.937500  1242.840745    497.90   1528.8000   
VOLTAS             6.0   2454.366667   951.226043   1231.20   1669.3125   
WHIRLPOOL          9.0   2279.438889  1009.094050   1446.10   1605.0000   
WIPRO              5.0   1273.660000   911.070205    573.95    651.7500   
YATHARTH           3.0   5277.183333  4857.601791    625.60   2757.0500   

                                             
                  50%         75%       max  
symbol                                       
3MINDIA     34890.625  35760.9625  36631.30  
AARTIDRUGS    583.300    583.3000    583.30  
AARTIIND     2172.200   4633.4375   6388.20  
AARTIPHARM    354.850    354.8500    354.85  
ALKYLAMINE   3326.400   3528.4000   4124.50  
...               ...         ...       ...  
VGUARD       2260.425   3061.7500   4356.00  
VOLTAS       3004.950   3026.5500   3227.25  
WHIRLPOOL    1894.000   2892.1000   4536.00  
WIPRO         651.750   1955.2500   2535.60  
YATHARTH     4888.500   7602.9750  10317.45  

[103 rows x 8 columns]

### Database

In [10]:
import sqlite3
from datetime import datetime
db_path = r'C:\Projects\Finances\database\Transactions.db'

In [11]:
connection = sqlite3.connect(db_path)
connection.execute("DELETE FROM zerodha")
zerodha_df.to_sql("zerodha",connection,if_exists="replace",index=False)
connection.commit()
connection.close()